# COVID-19 QC

Dans ce document, nous voulons utiliser le modèle [SIR](https://code-for-philly.gitbook.io/chime/what-is-chime/sir-modeling) développé par *Peen Medecine* et l'appliquer à la réalité du Québec.

In [29]:
import os
import altair as alt
import streamlit as st
from datetime import date

#os.chdir("../../..")
from src.penn_chime.charts import (
    build_admits_chart,
    build_census_chart,
    build_descriptions,
    build_sim_sir_w_date_chart,
    build_table,
)
from src.penn_chime.constants import DATE_FORMAT
from src.penn_chime.parameters import Parameters
from src.penn_chime.parameters import Parameters, Regions, Disposition

from src.penn_chime.models import SimSirModel
from src.penn_chime.settings import get_defaults
from src.penn_chime.presentation import (
    display_download_link,
    display_header,
    display_sidebar,
    hide_menu_style,
    write_definitions,
    write_footer,
)

In [45]:
# Parametres propres au Quebec

QC_POPULATION = 8500000
QC_MARKET_SHARE = 1 # Couverture du systeme de santé

QC_DATE_PREMIER_HOSP = '2019-03-15'

QC_NB_JOURS_HOSPITALISATION = 10
QC_NB_JOURS_SI = 12

QC_POURC_HOSPITALISATION = 0.08
QC_POURC_SI = 0.02
QC_POURC_VENTILATEUR = QC_POURC_SI

In [51]:
# Change the default settings
params = Parameters(
        population=QC_POPULATION,
        current_hospitalized=69,
        date_first_hospitalized=date(2020,3,7),
        doubling_time=4.0,
        hospitalized=Disposition(QC_POURC_HOSPITALISATION, QC_NB_JOURS_HOSPITALISATION),
        icu=Disposition(QC_POURC_SI, QC_NB_JOURS_SI),
        infectious_days=14,
        market_share=QC_MARKET_SHARE,
        n_days=200,
        mitigation_date=date.today(),
        relative_contact_rate=0.3,
        ventilated=Disposition(QC_POURC_VENTILATEUR, QC_NB_JOURS_SI),
    )

In [52]:
# Run the Penn Med SIR model
p = display_sidebar(st, params)
m = SimSirModel(p)

2020-04-05 10:13:48,830 - src.penn_chime.models - INFO - Using doubling_time: 4.0


Using doubling_time: 4.0


2020-04-05 10:13:48,871 - src.penn_chime.models - INFO - Set i_day = 24


Set i_day = 24


2020-04-05 10:13:48,874 - src.penn_chime.models - INFO - Estimated date_first_hospitalized: 2020-03-12; current_date: 2020-04-05; i_day: 24


Estimated date_first_hospitalized: 2020-03-12; current_date: 2020-04-05; i_day: 24


2020-04-05 10:13:48,876 - src.penn_chime.models - INFO - len(np.arange(-i_day, n_days+1)): 225


len(np.arange(-i_day, n_days+1)): 225


2020-04-05 10:13:48,878 - src.penn_chime.models - INFO - len(raw_df): 225


len(raw_df): 225


In [49]:
# Graphique nouveaux cas

admits_chart = build_admits_chart(alt=alt, admits_floor_df=m.admits_floor_df, max_y_axis=p.max_y_axis)
admits_chart

alt.LayerChart(...)

In [53]:
# Graphique cumulatif

census_chart = build_census_chart(alt=alt, census_floor_df=m.census_floor_df, max_y_axis=p.max_y_axis)
census_chart

alt.LayerChart(...)

In [54]:
# Créer le graphique retrospectif

m.census_floor_df['reel'] = 

,day,date,hospitalized,icu,ventilated
0,-24,2020-03-12,NaN,NaN,NaN
1,-23,2020-03-13,0.0,0.0,0.0
2,-22,2020-03-14,0.0,0.0,0.0
3,-21,2020-03-15,0.0,0.0,0.0
4,-20,2020-03-16,1.0,0.0,0.0
...,...,...,...,...,...
220,196,2020-10-18,240.0,87.0,87.0
221,197,2020-10-19,227.0,83.0,83.0
222,198,2020-10-20,215.0,78.0,78.0
223,199,2020-10-21,203.0,74.0,74.0
